#### Gathering data from database

requests; 

select createdat, deletedat, requestid, tsstart, type from calendar_event
 where createdat > '2025-03-01'
and type <> 'VACATION'

Qualitative info + target column

select r.id as requestid, 
zipregion, r.evaluationtime, r.desiredinstallationend, electricitybill, heatingbill l.mktgparamscore, a.netcontractsigned,  selfipaimportedat  -- for the two column that are present both in request and lead; request is more filled
from  request r left join lead l on l.id = r.leadid left join airtable_contracts a on a.requestid = r.id
-----NO TIME CONSTRAINT ON THIS -----

In [ ]:

import pandas as pd
import numpy as np


In [ ]:
# requests
file_path = r"C:\Users\ElifYilmaz\Downloads\project 2.0.csv"

df = pd.read_csv(file_path)
df.columns = df.columns.str.lower()


In [ ]:
# qualitative 

file2 = r"C:\Users\ElifYilmaz\Downloads\info client.csv"

df2 = pd.read_csv(file2)
df2.columns = df2.columns.str.lower()



In [ ]:
# merge for analysis

final_df = df.merge(df2, on='requestid', how='left')

Explatory checks

In [ ]:
# basic check, correlation can only be checked after encoding since most values are categorical and the rest are entries of happenings

# data types
print("Data types:")
print(final_df.dtypes)
print(f"\nAny missing values: {final_df.isnull().sum().sum()}")

# shape
print(f"Shape of dataset: {final_df.shape}")


In [ ]:
# Check missing values by column
print("Missing values per column:")
missing_counts = final_df.isnull().sum()
print(missing_counts[missing_counts > 0])

# Check the object columns that need encoding
object_cols = final_df.select_dtypes(include=['object']).columns
print(f"\nObject columns needing encoding: {list(object_cols)}")

In [ ]:
# categorical values
# Check unique values in categorical columns
for col in ['zipregion', 'evaluationtime', 'desiredinstallationend', 'mktgparamscore']:
    unique_count = final_df[col].nunique()
    print(f"{col}: {unique_count} unique values")
    if unique_count < 20:  # Show values if not too many
        print(f"  Values: {sorted(final_df[col].dropna().unique())}")
    print()

In [ ]:
# Check if missingness patterns relate to conversion
print("Conversion rates by missingness:")
for col in ['zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'mktgparamscore']:
    missing_conversion = final_df[final_df[col].isnull()]['netcontractsigned'].mean()
    present_conversion = final_df[final_df[col].notnull()]['netcontractsigned'].mean()
    
    print(f"{col}:")
    print(f"  Missing: {missing_conversion:.3f} ({final_df[col].isnull().sum()} cases)")
    print(f"  Present: {present_conversion:.3f} ({final_df[col].notnull().sum()} cases)")
    print()